# Named Entity Recognition
Theoretical explanation is present in this blogpost [link](). The project aims to use Bidirectional-LSTM with Conditional Random Fields along with static word embeddings to perform Named Entity Recognition task based off of the CoNLL-2003 dataset. This will be done step by step from downloading and preparing the dataset, embeddings, to training and validating the performance.

## 1. Dataset downloading and preprocessing
The first step is to download both the dataset which contains the tagged information as well as the static embeddings GloVe, which has vectors with embedded meanings for words. Since BiLSTM cannot generated contextualised vectors like BERT and other transformer models do (a natural improvement), we fallback to using GloVe.

### 1.1 Downloading GloVe Embeddings
We will download the 100 dimensional variant of GloVe (there's a 300 dimensional variant which is heavy but more accurate). This is available for your exploration at [Stanford NLP Github](https://github.com/stanfordnlp/GloVe/releases). The following script uses utility function to download this automatically and unzips to `src/data` folder.

In [1]:
# Common Imports
import numpy as np

In [2]:
from util.download import download_glove_embeddings, download_conll2003_dataset

In [3]:
download_glove_embeddings()

GloVe embeddings already exist. Skipping download.


### 1.2 Downloading the CoNLL2003 Dataset
The following method downloads the CoNLL2003 NER dataset in the same location as before.

In [4]:
download_conll2003_dataset()

CoNLL-2003 dataset already exists. Skipping download.


### 1.3 Loading the GloVe embeddings, CoNLL Data and create the dataset loader

In [5]:
from util.data import load_glove_embeddings, load_conll_dataset
from util.dataset import create_data_loader
from util.adapters import bio_tag_dictionary

In [6]:
# Load the GloVe embeddings
word2idx, embeddings, dimensions = load_glove_embeddings()

# Load the CoNLL as data
train_data, test_data, val_data = load_conll_dataset()
# Use the Train data as sample to get the tag dictionary
tag2label, label2tag = bio_tag_dictionary(train_data)

Loading GloVe embeddings from file.
Loaded 400002 words into vocabulary from GloVe.
Loading CoNLL data from data/conll2003/train.txt
Dataset loaded.
Loading CoNLL data from data/conll2003/test.txt
Dataset loaded.
Loading CoNLL data from data/conll2003/valid.txt
Dataset loaded.


### 1.4 Sanity Checks 
To ensure data has loaded in the format we expect, to prevent unnecessary bugs.

In [7]:
assert dimensions == 100
assert word2idx["the"] == 2
np.testing.assert_allclose(
    embeddings[word2idx["the"]],
    np.array([-0.038194, -0.24487, 0.72812, -0.39961, 0.083172, 0.043953, -0.39141, 0.3344, -0.57545, 0.087459, 0.28787, -0.06731, 0.30906, -0.26384, -0.13231, -0.20757, 0.33395, -0.33848, -0.31743, -0.48336, 0.1464, -0.37304, 0.34577, 0.052041, 0.44946, -0.46971, 0.02628, -0.54155, -0.15518, -0.14107, -0.039722, 0.28277, 0.14393, 0.23464, -0.31021, 0.086173, 0.20397, 0.52624, 0.17164, -0.082378, -0.71787, -0.41531, 0.20335, -0.12763, 0.41367, 0.55187, 0.57908, -0.33477,
              -0.36559, -0.54857, -0.062892, 0.26584, 0.30205, 0.99775, -0.80481, -3.0243, 0.01254, -0.36942, 2.2167, 0.72201, -0.24978, 0.92136, 0.034514, 0.46745, 1.1079, -0.19358, -0.074575, 0.23353, -0.052062, -0.22044, 0.057162, -0.15806, -0.30798, -0.41625, 0.37972, 0.15006, -0.53212, -0.2055, -1.2526, 0.071624, 0.70565, 0.49744, -0.42063, 0.26148, -1.538, -0.30223, -0.073438, -0.28312, 0.37104, -0.25217, 0.016215, -0.017099, -0.38984, 0.87424, -0.72569, -0.51058, -0.52028, -0.1459, 0.8278, 0.27062]),
    rtol=1e-5,
    atol=1e-8
)

In [8]:
train_data[0].get()

(['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'],
 ['b-org', 'o', 'b-misc', 'o', 'o', 'o', 'b-misc', 'o', 'o'])

In [9]:
BATCH_SIZE=64

# Create the actual dataloader
train_data_loader = create_data_loader(
    train_data, word2idx, tag2label, embeddings, batch_size=BATCH_SIZE)
val_data_loader = create_data_loader(
    val_data, word2idx, tag2label, embeddings, batch_size=BATCH_SIZE, 
    is_train=False)
test_data_loader = create_data_loader(
    test_data, word2idx, tag2label, embeddings, batch_size=BATCH_SIZE, 
    is_train=False)

## 2. Creating the Runner Instance
To run the training and validation loop.

In [10]:
from runner import Runner

In [11]:
# Defining constants here
LEARNING_RATE = 1e-4

In [12]:
runner = Runner(LEARNING_RATE, train_data_loader, val_data_loader,
                test_data_loader, dimensions, len(tag2label.keys()), 30, "", label2tag)

### 2.1. Training Loop

In [13]:
runner.train()

Training progress:   0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /pytorch/aten/src/ATen/native/TensorCompare.cpp:611.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch: 1/30, Training loss: 824.00, Validation loss: 574.81
Epoch: 2/30, Training loss: 474.36, Validation loss: 471.90
Epoch: 3/30, Training loss: 399.43, Validation loss: 364.60
Epoch: 4/30, Training loss: 316.81, Validation loss: 306.24
Epoch: 5/30, Training loss: 283.48, Validation loss: 284.55
Epoch: 6/30, Training loss: 262.83, Validation loss: 263.40
Epoch: 7/30, Training loss: 247.42, Validation loss: 253.70
Epoch: 8/30, Training loss: 233.79, Validation loss: 239.46
Epoch: 9/30, Training loss: 224.24, Validation loss: 230.30
Epoch: 10/30, Training loss: 214.34, Validation loss: 226.55
Epoch: 11/30, Training loss: 206.84, Validation loss: 219.09
Epoch: 12/30, Training loss: 200.87, Validation loss: 218.22
Epoch: 13/30, Training loss: 194.44, Validation loss: 206.64
Epoch: 14/30, Training loss: 188.77, Validation loss: 203.49
Epoch: 15/30, Training loss: 184.25, Validation loss: 199.04
Epoch: 16/30, Training loss: 179.20, Validation loss: 196.46
Epoch: 17/30, Training loss: 175.

### 2.2. Testing with test data set

In [17]:
filename = "bilstm_crf_20250519_121223_loss_182.58.pth"

In [18]:
runner = Runner(LEARNING_RATE, train_data_loader, val_data_loader,
                test_data_loader, dimensions, len(tag2label.keys()), 30, filename, label2tag)

In [19]:
runner.test()

Confusion Matrix:
[[ 1425    17   131    13     2     0     7     5    68]
 [   21   445    26    10     0     3     0     9   188]
 [  145    60  1163    18     0     1    23     4   247]
 [   35     4    33  1295     0     0     5   130   115]
 [   82     7     6     3    84     0    21     6    48]
 [    2    18     2     1     1    88     3     2    99]
 [  115     9    89    10    18     1   269    13   311]
 [   18     1    22   109     2     0     2   827   175]
 [   43    59    91    34     5    38    77    19 37957]]
Classification Report:
              precision    recall  f1-score   support

       b-loc       0.76      0.85      0.80      1668
      b-misc       0.72      0.63      0.67       702
       b-org       0.74      0.70      0.72      1661
       b-per       0.87      0.80      0.83      1617
       i-loc       0.75      0.33      0.46       257
      i-misc       0.67      0.41      0.51       216
       i-org       0.66      0.32      0.43       835
       i-per